# APP

In [11]:
import pickle
from PIL import Image
import numpy as np
from pathlib import Path
from epb_detection_system import preprocess_epb_image, EPBRecognitionSystem, TwoDPCA

In [12]:
# 1. Carrega o modelo
print("📦 Carregando modelo treinado...")
with open('models/epb_recognition_model.pkl', 'rb') as f:
    model_data = pickle.load(f)

system = model_data['system']
target_size = model_data['target_size']
print("   ✓ Modelo carregado com sucesso!\n")

# 2. Processa todas as imagens do diretório img-teste
img_folder = Path('img-teste')
image_files = sorted(list(img_folder.glob('*.png')) + list(img_folder.glob('*.jpg')) + list(img_folder.glob('*.jpeg')))

if len(image_files) == 0:
    print("⚠️  Nenhuma imagem encontrada no diretório 'img-teste'")
else:
    print("="*70)
    print(f"🔍 PREDIÇÕES PARA {len(image_files)} IMAGENS")
    print("="*70 + "\n")
    
    for img_path in image_files:
        try:
            # Carrega e processa imagem
            img = Image.open(img_path).convert('L')
            img = np.array(img.resize(target_size))
            img = preprocess_epb_image(img)
            
            # Faz predição
            prob = system.predict_proba(img.reshape(1, *img.shape))[0]
            pred = system.predict(img.reshape(1, *img.shape))[0]
            
            # Exibe resultado
            print(f"📸 {img_path.name}")
            print(f"   EPB detectada: {'SIM' if pred == 1 else 'NÃO'}")
            print(f"   Probabilidade: {prob[1]*100:.2f}%")
            print()
            
        except Exception as e:
            print(f"❌ {img_path.name}")
            print(f"   Erro ao processar: {str(e)}")
            print()
    
    print("="*70)
    print("✅ PROCESSAMENTO CONCLUÍDO")
    print("="*70)

📦 Carregando modelo treinado...
   ✓ Modelo carregado com sucesso!

🔍 PREDIÇÕES PARA 3 IMAGENS

📸 ara_com.png
   EPB detectada: SIM
   Probabilidade: 97.02%

📸 bjl_sem-2.png
   EPB detectada: NÃO
   Probabilidade: 0.09%

📸 bjl_sem.png
   EPB detectada: SIM
   Probabilidade: 50.17%

✅ PROCESSAMENTO CONCLUÍDO
